# Test full calculation
Test the full calculation of the 

In [1]:
pip install xmip

Note: you may need to restart the kernel to use updated packages.


## Import packages

In [2]:
%matplotlib inline
import xarray as xr
from xmip.preprocessing import rename_cmip6
import s3fs
import zarr

import util

## Summary of some test data

| model name | resolution | years in file for one variable | file size (one variable) | combined 3D variable data size | combined 2D variable data size |
| --- | --- | --- | --- | --- | --- |
| ACCESS-OM2 | 100 km | 10 | 670 MB | 17 GB | 0.4 GB |
| ACCESS-OM2-025 | 25 km | 1 | 1.1 GB | 271 GB | 7 GB |
| CMCC-CM2-SR5 | 100 km | 61 | 3.6 GB | 15 GB | 0.4 GB |
| CMCC-CM2-HR4 | 25 km | 10 | 9.4 GB | 231 GB | 6 GB |

## Load data from ESGF

In [3]:
result = util.esgf_search(activity_id='OMIP', experiment_id='omip2', variable_id='vmo', source_id='CMCC-CM2-SR5')

https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=CMCC-CM2-SR5&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0


In [4]:
# pick one file for testing
files_to_open = [result[-1]]

In [5]:
ds = xr.open_mfdataset(files_to_open, combine='by_coords')

## Pre-proc with `XMIP`

In [6]:
ds = rename_cmip6(ds)

In [9]:
ds

<xarray.Dataset>
Dimensions:      (time: 732, bnds: 2, lev: 50, y: 291, x: 360, vertex: 4)
Coordinates:
  * time         (time) object 1958-01-16 12:00:00 ... 2018-12-16 12:00:00
  * lev          (lev) float64 0.5126 1.621 2.858 ... 5.498e+03 5.904e+03
  * y            (y) int32 0 1 2 3 4 5 6 7 8 ... 283 284 285 286 287 288 289 290
  * x            (x) int32 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
Dimensions without coordinates: bnds, vertex
Data variables:
    time_bounds  (time, bnds) object dask.array<chunksize=(732, 2), meta=np.ndarray>
    lev_bounds   (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat_bounds   (y, x, vertex) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    lon_bounds   (y, x, vertex) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    lat          (y, x) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    vmo          (time, lev, y, x) float32 dask.array<chunksize=(732, 50, 291, 360), meta=np.ndarray>
Attributes: (12/39)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    comment:                         Ocean initial conditions: WOA 2013 T & S...
    contact:                         Pier Giuseppe Fogli (piergiuseppe.fogli@...
    creation_date:                   2020-02-19T13:20:13Z
    data_specs_version:              01.00.31
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    license:                         CMIP6 model data produced by CMCC is lic...
    cmor_version:                    3.5.0
    tracking_id:                     hdl:21.14100/119a9ea0-a62e-46ac-804d-879...
    DODS_EXTRA.Unlimited_Dimension:  time

In [11]:
def add_geo(DS):
    # assign geolat and geolon from ocean static
    DS = DS.assign_coords({'lon':DS['lon'].sel(x=DS.x,y=DS.y),
                           'lat':DS['lat'].sel(x=DS.x,y=DS.y)})
    return DS

In [13]:
ds = add_geo(ds)

In [15]:
ds

<xarray.Dataset>
Dimensions:      (time: 732, bnds: 2, lev: 50, y: 291, x: 360, vertex: 4)
Coordinates:
  * time         (time) object 1958-01-16 12:00:00 ... 2018-12-16 12:00:00
  * lev          (lev) float64 0.5126 1.621 2.858 ... 5.498e+03 5.904e+03
  * y            (y) int32 0 1 2 3 4 5 6 7 8 ... 283 284 285 286 287 288 289 290
  * x            (x) int32 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
    lat          (y, x) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
Dimensions without coordinates: bnds, vertex
Data variables:
    time_bounds  (time, bnds) object dask.array<chunksize=(732, 2), meta=np.ndarray>
    lev_bounds   (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat_bounds   (y, x, vertex) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    lon_bounds   (y, x, vertex) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    vmo          (time, lev, y, x) float32 dask.array<chunksize=(732, 50, 291, 360), meta=np.ndarray>
Attributes: (12/39)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    comment:                         Ocean initial conditions: WOA 2013 T & S...
    contact:                         Pier Giuseppe Fogli (piergiuseppe.fogli@...
    creation_date:                   2020-02-19T13:20:13Z
    data_specs_version:              01.00.31
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    license:                         CMIP6 model data produced by CMCC is lic...
    cmor_version:                    3.5.0
    tracking_id:                     hdl:21.14100/119a9ea0-a62e-46ac-804d-879...
    DODS_EXTRA.Unlimited_Dimension:  time

## Subset Pan-Arctic (lat>50N)

In [16]:
cond = (ds['lat']<=50)
dsnow = ds[['vmo']].where(cond,drop=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [32]:
dsnow = dsnow.chunk(chunks={'x':25,'y':25})

In [33]:
from dask_gateway import Gateway
gateway = Gateway()

In [34]:
##A line of trick to clean your dask cluster before you start your computation
from dask.distributed import Client
clusters=gateway.list_clusters()
print(clusters )
for cluster in clusters :
    cluster= gateway.connect(cluster.name)
    print(cluster)
    client = Client(cluster)
    client.close()
    cluster.shutdown()

[]


### Create a new Dask cluster with the Dask Gateway

In [38]:
cluster = gateway.new_cluster(worker_memory=2, worker_cores=1)

cluster.scale(20)
cluster

### Get a client from the Dask Gateway Cluster

As stated above, creating a Dask `Client` is mandatory in order to perform following Daks computations on your Dask Cluster.

In [39]:
from distributed import Client

if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /jupyterhub/services/dask-gateway/clusters/daskhub.a7a8b0e7b11f4a4e8d88f4500354d1e3/status,


In [23]:
your_name='test'
path='WAFFLES/'+your_name
s3_prefix =  "s3://"+path
print(s3_prefix)
access_key = !aws configure get aws_access_key_id
access_key = access_key[0]
secret_key = !aws configure get aws_secret_access_key
secret_key = secret_key[0]
client_kwargs={'endpoint_url': 'https://object-store.cloud.muni.cz'}

s3://WAFFLES/test


In [26]:
zarr_file_name= "FSStore_zarr"
uri = f"{s3_prefix}/{zarr_file_name}"
store = zarr.storage.FSStore(uri, client_kwargs=client_kwargs,key=access_key, secret=secret_key)
store.map.fs.find(uri)

[]

In [40]:
%time dsnow.to_zarr(store=store, mode='w', consolidated=True)

RuntimeError: NetCDF: Authorization failure

In [20]:
target = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })

In [21]:
s3_prefix =  "s3://WAFFLES"

## Extract Berring Strait

## Calc Volume Transport